# 锁

Redis 常在分布式环境中充当锁的作用，来保证特定资源在任何时候只能有一个使用者。

### 1. 普通锁

每个锁程序至少需要实现以下两个方法：
+ 加锁：尝试获得锁的独占权，在任何时候只能有最多一个客户端成功加锁，而除此以外的其他客户端则会失败。
+ 解锁：成功加锁的客户端可以通过解锁释放对锁的独占权，使包括它自身在内的所有客户端都能够有重新获得锁的机会。

在 Redis 中实现锁最基本的方法就是使用字符串数据结构，通过 `SET key value NX` 即可加锁成功。它的原理就是 `NX` 可以保证只有当前 key 不存在的时候才能创建成功，所以可以通过有没有成功创建该键，来判断有没有成功拿到锁。如果成功拿到锁，可以继续接下来的操作。

另外，带 NX 选项的 SET 命令是原子命令，所以如果有多个客户端同时进行上锁，也只会有一个客户端能够成功执行设置操作，是线程安全的。

当客户端需要解锁的时候，只需要使用 DEL 命令将该键删除即可。

In [13]:
"""
配置连接
"""
from redis import Redis

# Redis连接配置
client = Redis(
    host='39.104.208.122', 
    port=6379,
    decode_responses=True,
    ssl=False
)

if client.ping():
    print("Redis连接成功")
else:
    print("Redis连接失败")

Redis连接成功


In [5]:
class Lock:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def acquire(self):
        """
        尝试获取锁，成功时返回True，失败时则返回False。
        """
        # SET key value NX
        return self.client.set(self.key, "VALUE_OF_LOCK", nx=True) is True

    def release(self):
        """
        尝试释放锁，成功时返回True，失败时则返回False。
        """
        # DEL key
        return self.client.delete(self.key) == 1

In [6]:
locker1 = Lock(client, "Lock:10086")
# 模拟另一个客户端
locker2 = Lock(client, "Lock:10086")

print(locker1.acquire())
print(locker2.acquire())
print(locker1.release())
print(locker2.acquire())

True
False
True
True


### 2. 带自动解锁功能的锁

有一个问题，如果一个客户端拿到锁后，之后的执行过程非正常退出，那么这个锁永远都得不到释放，别的客户端永远拿不到锁。为了解决这个问题，可以给锁加上自动解锁的功能。这样如果一个客户端因为一些原因非正常退出，别的线程也可以拿到锁。

我们可以通过 `SET key value NX EX sec` 来设置该锁的最大存活时间，在指定的时限到达后，自动删除该键，从而解锁。

另外，在使用自动解锁时，锁的最大加锁时长必须超过程序在正常情况下完成任务操作的**最大时长**，比如 1s 可以完成任务，那么锁的过期时长应该设置为 30s 甚至更长，以免程序出现适当延误，而直接剥夺锁的使用权，导致任务失败。

换句话说，锁的自动解锁功能就跟程序异常一样，是一种保护措施。成功加锁的客户端在程序正常运行的情况下还是应该手动解锁，而不是依靠自动解锁。

In [16]:
class AutoReleaseLock:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def acquire(self, timeout, unit="sec"):
        """
        尝试获取一个能够在指定时长之后自动释放的锁。
        timeout参数用于设置锁的最大加锁时长。
        可选的unit参数则用于设置时长的单位，它的值可以是代表秒的'sec'或是代表毫秒的'ms'，默认为'sec'。
        """
        if unit == "sec":
            # SET key value NX EX sec
            return self.client.set(self.key, "VALUE_OF_LOCK", nx=True, ex=timeout) is True
        elif unit == "ms":
            # SET key value NX PX ms
            return self.client.set(self.key, "VALUE_OF_LOCK", nx=True, px=timeout) is True
        else:
            raise ValueError("Unit must be 'sec' or 'ms'!")

    def release(self):
        """
        尝试释放锁，成功时返回True，失败时则返回False。
        """
        return self.client.delete(self.key) == 1

In [15]:
import time

locker1 = AutoReleaseLock(client, "Lock:10087")
# 模拟另一个客户端
locker2 = AutoReleaseLock(client, "Lock:10087")

print(locker1.acquire(5))
time.sleep(5)
print(locker2.acquire(5))

True
True
